## Function to read json rows into a dataframe
json_df = pd.DataFrame.from_records(df['edp_raw_data_map'].apply(json.loads))

In [1]:
import  plotly.express as px
import pandas as pd
import pyodbc
import json
from datetime import datetime 

def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')
pd.options.display.max_columns = 100

# Here is a function I use to read a file with a sql query
def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

# Then I use this to run the query and get the results as a dataframe
def run_query(query, dsn='edp-workbench-cshub'):
    cnxn = pyodbc.connect('DSN='+dsn, autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df


In [ ]:
df = run_query(file_to_string('SQL/EfficiencyMetrics.SQL'))

In [9]:
df =run_query(
"""
SELECT *,element_at(edp_raw_data_map, 'ExtraData_totalTime'),element_at(VHE.edp_raw_data_map, 'Identities_messageSid')
FROM hive.care.l1_verizon_home_events vhe
where 
    edp_updated_date = '2023-09-26' and 
    element_at(edp_raw_data_map, '_header_eventContext_producer') = 'eip-ingestion-data-science' and
    element_at(edp_raw_data_map, 'Identities_messageSid') is not null and
    element_at(edp_raw_data_map, 'Name') = 'RequestSummaryVoice'
    
limit 1000
""")

display(df.head())
json_df =  pd.DataFrame.from_records(df['edp_raw_data_map'].copy().apply(json.loads))
display(json_df.head())
display(json_df[['ExtraData_totalTime']].head())
cols = json_df.columns.copy()
cols.sort_values()
json_df[cols].head()

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_16012\1418089109.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,batchid,edp_raw_data,edp_headers,meta_kafka_metadata,eventid,op,edp_updated_date,dms_time_stamp,edp_raw_data_map,_col9,_col10
0,54962,"{""Name"":""RequestSummaryVoice"",""Scope"":""Generat...","{""/encryption/AES_KMS/encryptedKey"":""\u0001\u0...","[""platform.ingest.app.home-events"",1,302439086...",497567964,I,2023-09-26,1695771315534,"{""ExtraData_totalTime"":""3.8022686690001137"",""_...",3.8022686690001137,349e81b9-01fc-49c4-89e7-6c052ebdf61d
1,54962,"{""Name"":""RequestSummaryVoice"",""Scope"":""Generat...","{""/encryption/AES_KMS/encryptedKey"":""\u0001\u0...","[""platform.ingest.app.home-events"",3,302440269...",1994435431,I,2023-09-26,1695771315787,"{""ExtraData_totalTime"":""2.7861030620006204"",""_...",2.7861030620006204,67417a9d-00cd-40c8-b16a-53f7eceff8c8
2,54962,"{""Name"":""RequestSummaryVoice"",""Scope"":""Generat...","{""/encryption/AES_KMS/encryptedKey"":""\u0001\u0...","[""platform.ingest.app.home-events"",3,302443174...",872528317,I,2023-09-26,1695771315826,"{""ExtraData_totalTime"":""3.334132605999912"",""_h...",3.334132605999912,2a36c6ec-4134-42c6-aa78-f453b9c202d3
3,54962,"{""Name"":""RequestSummaryVoice"",""Scope"":""Generat...","{""/encryption/AES_KMS/encryptedKey"":""\u0001\u0...","[""platform.ingest.app.home-events"",2,302427564...",-16827371,I,2023-09-26,1695771327382,"{""ExtraData_totalTime"":""1.994491477000338"",""_h...",1.994491477000338,0b5af463-5375-433a-9bf2-7f036334a97b
4,54962,"{""Name"":""RequestSummaryVoice"",""Scope"":""Generat...","{""/encryption/AES_KMS/encryptedKey"":""\u0001\u0...","[""platform.ingest.app.home-events"",2,302425264...",810664310,I,2023-09-26,1695771319682,"{""ExtraData_totalTime"":""3.921798305000266"",""_h...",3.921798305000266,5708f6d3-32af-42ab-a5df-ac9c8df004b2


,ExtraData_totalTime,_header_client,Identities_callSid,MetaData_RequestContext_Host,MetaData_RequestContext_User-Agent,ExtraData_openAITime,Name,_header_eventContext_producer,ExtraData_isLive,_header_category,_header_eventId,MetaData_RequestContext_Accept,_header_domain,_header_schema,EventId,_header_object,ExtraData_agentId,_header_eventContext_category,MetaData_RequestContext_Accept-Encoding,ExtraData_botResponse,_header_eventContext_sourceType,ExtraData_inputMessage,ExtraData_helixTime,_header_eventContext_region,_header_eventContext_ingestionTimestamp,_header_eventContext_endpoint,ExtraData_promptTranscript,_header_source,MetaData_RequestContext_X-Forwarded-For,ExtraData_ozmoTime,ExtraData_Time,MetaData_ServerTime,MetaData_Client,MetaData_RequestContext_X-Forwarded-Proto,ExtraData_endRequestTime,Time,_header_tags,_header_timestamp,ExtraData_transcriptsTime,MetaData__eventType,MetaData_RequestContext_X-Amzn-Trace-Id,MetaData_ProcessorType,ExtraData_totalTokens,MetaData_IsTest,Identities_messageSid,ExtraData_embeddingsTime,ExtraData_startRequestTime,ExtraData_helixNewTime,_header_action,MetaData_RequestContext_Content-Type,Scope,FullName,MetaData_RequestContext_X-Forwarded-Port,_header_eventContext_correlationId,Identities_SessionId,ExtraData_deviceDetectionTime
0,3.8022686690001137,verizon,No_CallSid,ingestion.events.asurion.com,Python/3.10 aiohttp/3.8.5,3.081180687999222,RequestSummaryVoice,eip-ingestion-data-science,true,Analytics,387bd557-69bc-46c2-9d2d-b83d532d45ec,*/*,event,analytics.ingestion.v1,387bd557-69bc-46c2-9d2d-b83d532d45ec,GenerativeAISearchBotVoice,575731,analytics,"gzip, deflate","If your network extender is not activating, he...",HTTP,not activating,0.030364360000021406,us-east-1,1695770180092,ingestion,"[{""callSid"":""No_CallSid"",""content"":""step 11 d ...",EDaaS|Eventing|US,34.205.188.78,0,2023-09-26T23:16:20.092Z,2023-09-26T23:16:20.100328458Z,verizon,https,2023-09-26 18:16:20.053577,2023-09-26T23:16:20.092Z,[],1695770180100,0.05530893299874151,event,Root=1-65136644-7063bf0816eefcf41c144323,EIP,1729,false,349e81b9-01fc-49c4-89e7-6c052ebdf61d,0.48574388599990925,2023-09-26 18:16:16.238988,0.06988517699937802,created,application/json,GenerativeAISearchBotVoice,GenerativeAISearchBotVoice_RequestSummaryVoice,443,220d2d60-ced3-4921-9b3f-3428c0d0a0c2,b2dc7364-4dbd-45a8-a1b8-e9ee5483ef81,0.000260409999100375
1,2.7861030620006204,verizon,No_CallSid,ingestion.events.asurion.com,Python/3.10 aiohttp/3.8.5,2.079903544999979,RequestSummaryVoice,eip-ingestion-data-science,true,Analytics,7d96c350-08f9-4f3c-a09e-627866db9796,*/*,event,analytics.ingestion.v1,7d96c350-08f9-4f3c-a09e-627866db9796,GenerativeAISearchBotVoice,573192,analytics,"gzip, deflate","Okay, let's try resetting the network settings...",HTTP,yes,0.03296359299929463,us-east-2,1695768471287,ingestion,"[{""callSid"":""No_CallSid"",""content"":""galaxy s23...",EDaaS|Eventing|US,34.203.46.168,0,2023-09-26T22:47:51.287Z,2023-09-26T22:47:51.304571439Z,verizon,https,2023-09-26 17:47:51.246594,2023-09-26T22:47:51.287Z,[],1695768471304,0.04840254800001276,event,Root=1-65135f97-660ade4660f07b111de0922c,EIP,1711,false,67417a9d-00cd-40c8-b16a-53f7eceff8c8,0.4644443269990006,2023-09-26 17:47:48.447687,0.04149109399986628,created,application/json,GenerativeAISearchBotVoice,GenerativeAISearchBotVoice_RequestSummaryVoice,443,1c6bee25-9f72-40dc-8846-a0e08184e8f3,79876eba-fcfe-4747-b475-d863689f6333,0.004443159999937052
2,3.334132605999912,verizon,No_CallSid,ingestion.events.asurion.com,Python/3.10 aiohttp/3.8.5,2.4827243970012205,RequestSummaryVoice,eip-ingestion-data-science,true,Analytics,7fb24bb3-b5df-47ac-b2b9-dc3c3cf17355,*/*,event,analytics.ingestion.v1,7fb24bb3-b5df-47ac-b2b9-dc3c3cf17355,GenerativeAISearchBotVoice,573192,analytics,"gzip, deflate",I'm sorry to hear that the previous solution d...,HTTP,didnt work,0.03825017400049546,us-east-2,1695768880438,ingestion,"[{""callSid"":""No_CallSid"",""content"":""Galaxy s23...",EDaaS|Eventing|US,34.203.46.168,0,202

,ExtraData_totalTime
0,3.8022686690001137
1,2.7861030620006204
2,3.334132605999912
3,1.994491477000338
4,3.921798305000266


,ExtraData_totalTime,_header_client,Identities_callSid,MetaData_RequestContext_Host,MetaData_RequestContext_User-Agent,ExtraData_openAITime,Name,_header_eventContext_producer,ExtraData_isLive,_header_category,_header_eventId,MetaData_RequestContext_Accept,_header_domain,_header_schema,EventId,_header_object,ExtraData_agentId,_header_eventContext_category,MetaData_RequestContext_Accept-Encoding,ExtraData_botResponse,_header_eventContext_sourceType,ExtraData_inputMessage,ExtraData_helixTime,_header_eventContext_region,_header_eventContext_ingestionTimestamp,_header_eventContext_endpoint,ExtraData_promptTranscript,_header_source,MetaData_RequestContext_X-Forwarded-For,ExtraData_ozmoTime,ExtraData_Time,MetaData_ServerTime,MetaData_Client,MetaData_RequestContext_X-Forwarded-Proto,ExtraData_endRequestTime,Time,_header_tags,_header_timestamp,ExtraData_transcriptsTime,MetaData__eventType,MetaData_RequestContext_X-Amzn-Trace-Id,MetaData_ProcessorType,ExtraData_totalTokens,MetaData_IsTest,Identities_messageSid,ExtraData_embeddingsTime,ExtraData_startRequestTime,ExtraData_helixNewTime,_header_action,MetaData_RequestContext_Content-Type,Scope,FullName,MetaData_RequestContext_X-Forwarded-Port,_header_eventContext_correlationId,Identities_SessionId,ExtraData_deviceDetectionTime
0,3.8022686690001137,verizon,No_CallSid,ingestion.events.asurion.com,Python/3.10 aiohttp/3.8.5,3.081180687999222,RequestSummaryVoice,eip-ingestion-data-science,true,Analytics,387bd557-69bc-46c2-9d2d-b83d532d45ec,*/*,event,analytics.ingestion.v1,387bd557-69bc-46c2-9d2d-b83d532d45ec,GenerativeAISearchBotVoice,575731,analytics,"gzip, deflate","If your network extender is not activating, he...",HTTP,not activating,0.030364360000021406,us-east-1,1695770180092,ingestion,"[{""callSid"":""No_CallSid"",""content"":""step 11 d ...",EDaaS|Eventing|US,34.205.188.78,0,2023-09-26T23:16:20.092Z,2023-09-26T23:16:20.100328458Z,verizon,https,2023-09-26 18:16:20.053577,2023-09-26T23:16:20.092Z,[],1695770180100,0.05530893299874151,event,Root=1-65136644-7063bf0816eefcf41c144323,EIP,1729,false,349e81b9-01fc-49c4-89e7-6c052ebdf61d,0.48574388599990925,2023-09-26 18:16:16.238988,0.06988517699937802,created,application/json,GenerativeAISearchBotVoice,GenerativeAISearchBotVoice_RequestSummaryVoice,443,220d2d60-ced3-4921-9b3f-3428c0d0a0c2,b2dc7364-4dbd-45a8-a1b8-e9ee5483ef81,0.000260409999100375
1,2.7861030620006204,verizon,No_CallSid,ingestion.events.asurion.com,Python/3.10 aiohttp/3.8.5,2.079903544999979,RequestSummaryVoice,eip-ingestion-data-science,true,Analytics,7d96c350-08f9-4f3c-a09e-627866db9796,*/*,event,analytics.ingestion.v1,7d96c350-08f9-4f3c-a09e-627866db9796,GenerativeAISearchBotVoice,573192,analytics,"gzip, deflate","Okay, let's try resetting the network settings...",HTTP,yes,0.03296359299929463,us-east-2,1695768471287,ingestion,"[{""callSid"":""No_CallSid"",""content"":""galaxy s23...",EDaaS|Eventing|US,34.203.46.168,0,2023-09-26T22:47:51.287Z,2023-09-26T22:47:51.304571439Z,verizon,https,2023-09-26 17:47:51.246594,2023-09-26T22:47:51.287Z,[],1695768471304,0.04840254800001276,event,Root=1-65135f97-660ade4660f07b111de0922c,EIP,1711,false,67417a9d-00cd-40c8-b16a-53f7eceff8c8,0.4644443269990006,2023-09-26 17:47:48.447687,0.04149109399986628,created,application/json,GenerativeAISearchBotVoice,GenerativeAISearchBotVoice_RequestSummaryVoice,443,1c6bee25-9f72-40dc-8846-a0e08184e8f3,79876eba-fcfe-4747-b475-d863689f6333,0.004443159999937052
2,3.334132605999912,verizon,No_CallSid,ingestion.events.asurion.com,Python/3.10 aiohttp/3.8.5,2.4827243970012205,RequestSummaryVoice,eip-ingestion-data-science,true,Analytics,7fb24bb3-b5df-47ac-b2b9-dc3c3cf17355,*/*,event,analytics.ingestion.v1,7fb24bb3-b5df-47ac-b2b9-dc3c3cf17355,GenerativeAISearchBotVoice,573192,analytics,"gzip, deflate",I'm sorry to hear that the previous solution d...,HTTP,didnt work,0.03825017400049546,us-east-2,1695768880438,ingestion,"[{""callSid"":""No_CallSid"",""content"":""Galaxy s23...",EDaaS|Eventing|US,34.203.46.168,0,202

In [13]:
# test_df = run_query(query_file_to_string('SQL/ExpertAssist.SQL'))
test_df.head()
pivot = pd.pivot_table(test_df, index='Date', values=['TotalUses','BackendLatency','CallsWithUse'], aggfunc=sum)
pivot['AvgLatency']= pivot['BackendLatency']/pivot['TotalUses']
px.line(pivot)

In [11]:
for query in [
        'Sales.SQL',
        'HelixSearch.SQL',
        'EfficiencyMetrics.SQL',
        'CXP.SQL'
        ]:
    display(query+" "+timestamp())
    display(run_query(query_file_to_string('SQL/'+query)).head())

'Sales.SQL 2023-09-29 11:35:43 AM'

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_21196\85752655.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,Employee,Date,work_type,Total_Eligible,Total_Accepted
0,573276,2023-09-21,PPX,7.0,1.0
1,567923,2023-06-16,PPX,11.0,0.0
2,576911,2023-09-06,PPX,10.0,1.0
3,573573,2023-08-28,PPX,11.0,1.0
4,567923,2023-07-20,PPX,13.0,0.0


'HelixSearch.SQL 2023-09-29 11:36:01 AM'

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_21196\85752655.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


...

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_21196\85752655.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,Employee,Date,work_type,HoursWorked,Sessions,Total_Resolution_Time
0,557903,2023-07-14,PPX,8.02,9,13612
1,568438,2023-07-19,PPX,7.84,17,15541
2,557903,2023-09-19,PPX,8.20,10,15028
3,557903,2023-07-18,PPX,8.00,7,10871
4,557903,2023-07-06,PPX,8.03,13,16009


'CXP.SQL 2023-09-29 11:36:37 AM'

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_21196\85752655.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,Date,Employee,CXP_Prob_Sum,CXP_Prob_Count,work_type
0,2023-07-24,393510,6.702235,13,PPX
1,2023-06-26,552121,0.530424,1,PPX
2,2023-07-25,541226,15.232828,30,PPX
3,2023-07-10,554487,7.249915,14,PPX
4,2023-07-28,552121,2.605552,5,PPX


In [ ]:
run_query("SELECT * FROM hive.information_schema.tables where table_name like '%home%' and table_schema = 'care'")


In [ ]:
duplicateChecker = run_query(
"""
SELECT element_at(VHE.edp_raw_data_map, 'Identities_SessionId'), vsc.support_session_id
FROM
    hive.care.l1_verizon_home_events vhe
    LEFT OUTER JOIN
    hive.care.l3_verizon_soluto_crm vsc
    ON  element_at(VHE.edp_raw_data_map, 'Identities_SessionId') = vsc.support_session_id
WHERE 
    element_at(VHE.edp_raw_data_map, '_header_eventContext_producer') = 'eip-ingestion-data-science' and
    element_at(VHE.edp_raw_data_map, 'Identities_messageSid') is not null and
    element_at(VHE.edp_raw_data_map, 'Name') = 'RequestSummaryVoice'
""")

In [ ]:
display(duplicateChecker.columns)
duplicateChecker

In [50]:
run_query(
"""
SELECT
    element_at(VHE.edp_raw_data_map,'Identities_SessionId'),
    substring(element_at(VHE.edp_raw_data_map, 'ExtraData_endRequestTime'),-10) as "Date1",
    substring(element_at(VHE.edp_raw_data_map, 'ExtraData_endRequestTime'),1, 10) as "Date2",
    substring(element_at(VHE.edp_raw_data_map, 'ExtraData_endRequestTime'),+10) as "Date3"
FROM
     hive.care.l1_verizon_home_events VHE
WHERE 1=1
    AND element_at(VHE.edp_raw_data_map, '_header_eventContext_producer') = 'eip-ingestion-data-science'
    AND element_at(VHE.edp_raw_data_map, 'Identities_messageSid') IS NOT NULL
    AND element_at(VHE.edp_raw_data_map, 'Name') = 'RequestSummaryVoice'
""")

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_3772\1418089109.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,_col0,Date1,Date2,Date3
0,08c60eb5-f7e5-415d-8226-1212d7dad67e,:01.015052,2023-09-25,5 18:09:01.015052
1,13d7b6c4-8005-4833-bed8-f278115c3e38,:38.518442,2023-09-25,5 18:18:38.518442
2,bdf3f329-1548-4505-afd5-332c2fa1214c,:53.842888,2023-09-27,7 08:07:53.842888
3,bfe9aaaa-fc27-4fa1-8953-5307b4080b4b,:29.487274,2023-09-27,7 07:59:29.487274
4,c587e1f9-7461-4333-9e8e-04101101df36,:16.351127,2023-09-27,7 08:06:16.351127
...,...,...,...,...
1361,69ee8dc9-c717-4c2b-9a89-5c0bee5ff9d8,:32.294672,2023-09-26,6 17:48:32.294672
1362,1956ba0e-87f7-461d-9546-6ebe97f9db8a,:31.144620,2023-09-28,8 19:54:31.144620
1363,0d3ea442-c454-4e8f-9aa2-3ba568495d8d,:05.746993,2023-09-28,8 19:01:05.746993
1364,595e11f2-3976-473a-aa70-a15899cbd917,:33.121318,2023-09-28,8 19:01:33.121318
